In [ ]:
import sys
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from utils import *
from equations import *

In [ ]:
# Manually making sure the numpy random seeds are "the same" on all devices
np.random.seed(1234)
tf.random.set_seed(1234)

# Hyper parameters {{{
hp = {}
# Data size on the solution u
hp["N_u"] = 3000
# Collocation points size, where we’ll check for f = 0
hp["N_f"] = 500
# DeepNN topology (2-sized input [x t], 8 hidden layer of 20-width, 1-sized output [u]
hp["layers"] = [2, 20, 20, 20, 20, 20, 2]
hp["h_layers"] = [2, 20, 20, 20, 20, 20,2]
hp["C_layers"] = [2, 20, 20, 20, 20, 20, 1]
# Setting up the TF SGD-based optimizer (set tf_epochs=0 to cancel it)
hp["tf_epochs"] = 100000
hp["tf_lr"] = 0.001
hp["tf_b1"] = 0.99
hp["tf_eps"] = 1e-1
# Setting up the quasi-newton LBGFS optimizer (set nt_epochs=0 to cancel it)
hp["nt_epochs"] = 50000
hp["nt_lr"] = 0.8
hp["nt_ncorr"] = 50
hp["log_frequency"] = 1000
hp["use_tfp"] = True
# Record the history
hp["save_history"] = True
hp["history_frequency"] = 10
# path for loading data and saving models
repoPath = "./"
appDataPath = os.path.join(repoPath, "matlab_SSA", "DATA")
# path = os.path.join(appDataPath, "SSA2D_nocalving.mat")
# path = os.path.join(appDataPath, "SSA2D_seg_nocalving.mat")
path = os.path.join(appDataPath, "SSA2D_segCF.mat")
# path = os.path.join(appDataPath, "Helheim_Weertman_iT080_PINN_fastflow_CF.mat")
modelPath = "./Models/test"
reloadModel = False # reload from previous training
#}}}

In [ ]:
# load the data
x, y, Exact_vx, Exact_vy, X_star, u_star, X_u_train, u_train, X_f, X_bc, u_bc, X_cf, n_cf, xub, xlb, uub, ulb = prep_Helheim_data_all(path, hp["N_u"], hp["N_f"])

# Creating the model and training
logger = Logger(hp)
pinn = SSA3NN_calvingfront_invertC(hp, logger, X_f, 
        X_bc, u_bc,
        X_cf, n_cf,
        xub, xlb, uub, ulb, 
        modelPath, reloadModel,
        mu=1.1060e8,
        loss_weights=[1e-1, 1e-2, 1e-2, 1e-4, 1e-10])

X_u = pinn.tensor(X_star)
u = pinn.tensor(u_star)
# error function for logger
def error():
    return pinn.test_error(X_u, u)
logger.set_error_fn(error)

In [ ]:
# train the model
pinn.fit(X_u_train, u_train)

# save
#pinn.model.save("./Models/SSA2D_friction_1e_4_TF"+str(hp["tf_epochs"]) +"_NT"+str(hp["nt_epochs"]))

In [ ]:
# plot
vranges = {}
vranges['u - u obs']=[-20, 20]
vranges['v - v obs']=[-20, 20]
vranges['H - H obs']=[-20, 20]
vranges['h - h obs']=[-20, 20]
vranges['C - C obs']=[-500, 500]
vranges['C pred']=[0, 3500]
plot_Helheim_all(pinn, X_f, X_star, u_star, xlb, xub, vranges)

# history
plot_log_history(pinn)